In [1]:
%%sql
Lakehouse_

-- Modelando as tabela dimensões e fato, modelagem dimensional
CREATE OR REPLACE TABLE Lakehouse_Gold.fact_cotacoes
USING DELTA
AS
  
-- Mínimas datas por moeda
WITH base_dados AS (
SELECT
    c.Moeda,
    MIN(c.Data) AS min_data
FROM Lakehouse_Silver.cotacoes c
GROUP BY c.Moeda
),

-- Gera todas as datas do período
datas_periodo AS (
SELECT
    Moeda,
    explode(
        sequence(
            min_data,
            current_date(), --Data atual
            interval 1 day
        )
    ) AS Data
FROM base_dados
),

-- Mescla as cotações atuais com as datas do período
dados_completos AS (
SELECT
    dp.Moeda,
    dp.Data,
    COALESCE(c.Cotacao, NULL) AS Cotacao
FROM datas_periodo dp
LEFT JOIN Lakehouse_Silver.cotacoes c
ON dp.Moeda = c.Moeda 
    AND dp.Data = c.Data
),

-- Preenche as lacunas com a última cotação de cada moeda
fact_cotacoes_final AS (
SELECT
    Moeda,
    Data,
    last_value(Cotacao, true)
    OVER (
        PARTITION BY Moeda
        ORDER BY Data
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS Cotacao
FROM dados_completos
)

-- Saída
SELECT * FROM fact_cotacoes_final
ORDER BY Data, Moeda;

     
StatementMeta(, 8efee4c9-a34c-4f64-ba95-a53753b10db3, 2, Finished, Available, Finished)
<Spark SQL result set with 0 rows and 0 fields>


StatementMeta(, abdebb49-1b0a-4426-9720-a2d27db8a125, 3, Finished, , Finished)

<Spark SQL result set with 0 rows and 0 fields>

Error: 
[PARSE_SYNTAX_ERROR] Syntax error at or near 'StatementMeta'.(line 4, pos 0)

== SQL ==


     
StatementMeta(, 8efee4c9-a34c-4f64-ba95-a53753b10db3, 2, Finished, Available, Finished)
^^^
<Spark SQL result set with 0 rows and 0 fields>
